In [9]:
pip install snowflake-connector-python pandas python-dotenv cryptography

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from dotenv import load_dotenv
from cryptography.hazmat.primitives import serialization
import snowflake.connector
import pandas as pd

#Bloque exclusivo de importacion de librerias

In [11]:
load_dotenv('.env')

private_key_pem = os.getenv("SNOWFLAKE_PRIVATE_KEY").replace('\\n', '\n')

private_key_obj = serialization.load_pem_private_key(
    private_key_pem.encode('utf-8'),
    password=None,
)
private_key_der = private_key_obj.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

conn = None

try:
    conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    private_key=private_key_der,
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
    )
    print("Conectado a Snowflake exitosamente")
except Exception as e:
    print(f"No conectado por: {e}")


Conectado a Snowflake exitosamente


In [12]:
pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)

In [13]:
print("Pregunta de Negocio 1: Demanda por zona y mes: ¿cuáles son las 10 zonas con más viajes por mes? (Pickup Zone)")

# query para zonas más demandadas por mes en pickup
query = """
with viajes_por_zona as (
    select 
        t.pickup_zone_sk,
        dz.pickup_zone,
        dd.month as mes,
        count(*) as total_de_viajes
    from taxis_gold.fct_trips as t
    join taxis_gold.dim_zone_pickup as dz using (pickup_zone_sk)
    join taxis_gold.dim_date_pickup as dd using (pickup_date_sk)
    group by t.pickup_zone_sk, dz.pickup_zone, dd.month
),

top10_por_mes as (
    select v1.*
    from viajes_por_zona v1
    where (
        select count(*)
        from viajes_por_zona v2
        where v2.mes = v1.mes
          and v2.total_de_viajes > v1.total_de_viajes
    ) < 10
)

select *
from top10_por_mes
order by mes, total_de_viajes desc;
"""

# Ejecutar query y devolver DataFrame
df = pd.read_sql(query, conn)

# Ver resultados
print(df)

Pregunta de Negocio 1: Demanda por zona y mes: ¿cuáles son las 10 zonas con más viajes por mes? (Pickup Zone)


C:\Users\john8\AppData\Local\Temp\ipykernel_25788\3884383967.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


     PICKUP_ZONE_SK                   PICKUP_ZONE  MES  TOTAL_DE_VIAJES
0               237         Upper East Side South    1          2834868
1               236         Upper East Side North    1          2714159
2               161                Midtown Center    1          2664068
3               162                  Midtown East    1          2323551
4               230     Times Sq/Theatre District    1          2293955
5               186  Penn Station/Madison Sq West    1          2280393
6               234                      Union Sq    1          2133446
7               170                   Murray Hill    1          2109537
8                48                  Clinton East    1          2046545
9               142           Lincoln Square East    1          2024501
10              237         Upper East Side South    2          2707492
11              161                Midtown Center    2          2621065
12              236         Upper East Side North    2          

In [14]:
print("Pregunta de Negocio 1: Demanda por zona y mes: ¿cuáles son las 10 zonas con más viajes por mes? (Dropoff Zone)")


# query para zonas más demandadas por mes en dropoff
query = """
with viajes_por_zona as (
    select 
        t.dropoff_zone_sk,
        dz.dropoff_zone,
        dd.month as mes,
        count(*) as total_de_viajes
    from taxis_gold.fct_trips as t
    join taxis_gold.dim_zone_dropoff as dz using (dropoff_zone_sk)
    join taxis_gold.dim_date_dropoff as dd using (dropoff_date_sk)
    group by t.dropoff_zone_sk, dz.dropoff_zone, dd.month
),

top10_por_mes as (
    select v1.*
    from viajes_por_zona v1
    where (
        select count(*)
        from viajes_por_zona v2
        where v2.mes = v1.mes
          and v2.total_de_viajes > v1.total_de_viajes
    ) < 10
)

select *
from top10_por_mes
order by mes, total_de_viajes desc;
"""

# Ejecutar query y devolver DataFrame
df = pd.read_sql(query, conn)

# Ver resultados
print(df)

Pregunta de Negocio 1: Demanda por zona y mes: ¿cuáles son las 10 zonas con más viajes por mes? (Dropoff Zone)


C:\Users\john8\AppData\Local\Temp\ipykernel_25788\143461761.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


     DROPOFF_ZONE_SK                  DROPOFF_ZONE  MES  TOTAL_DE_VIAJES
0                236         Upper East Side North    1          2861884
1                237         Upper East Side South    1          2536732
2                161                Midtown Center    1          2528088
3                170                   Murray Hill    1          2132406
4                230     Times Sq/Theatre District    1          2031040
5                162                  Midtown East    1          2015201
6                142           Lincoln Square East    1          1855314
7                234                      Union Sq    1          1853306
8                 48                  Clinton East    1          1824844
9                239         Upper West Side South    1          1791122
10               236         Upper East Side North    2          2691543
11               161                Midtown Center    2          2490805
12               237         Upper East Side South 

In [15]:
print("Pregunta de Negocio 2: Ingresos y propinas: ¿cómo varían los ingresos totales y el tip % por borough y mes? (Acorde a la zona de Pickup)")

# query para ingresos y propinas por borough y mes en pickup
query = """
with viajes_con_borough as (
    select 
        t.fare_amount,
        t.tip_percentage,
        dd.month as mes,
        dz.pickup_borough
    from ny_taxi.taxis_gold.fct_trips as t
    join ny_taxi.taxis_gold.dim_zone_pickup as dz using (pickup_zone_sk)
    join ny_taxi.taxis_gold.dim_date_pickup as dd using (pickup_date_sk)
    where t.fare_amount >= 0 and dz.pickup_borough is not null
)

select
    pickup_borough as pickup_borough,
    mes,
    sum(fare_amount) as ingresos_totales,
    avg(tip_percentage) as tip_porcentaje_promedio
from viajes_con_borough
group by pickup_borough, mes
order by pickup_borough, mes;
"""

# Ejecutar query y devolver DataFrame
df = pd.read_sql(query, conn)

# Ver resultados
print(df)

Pregunta de Negocio 2: Ingresos y propinas: ¿cómo varían los ingresos totales y el tip % por borough y mes? (Acorde a la zona de Pickup)


C:\Users\john8\AppData\Local\Temp\ipykernel_25788\1764391865.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   PICKUP_BOROUGH  MES  INGRESOS_TOTALES  TIP_PORCENTAJE_PROMEDIO
0           Bronx    1      5.313130e+06                 3.183229
1           Bronx    2      5.777583e+06                 2.997751
2           Bronx    3      6.742380e+06                 7.853459
3           Bronx    4      5.882890e+06                 3.397656
4           Bronx    5      6.078075e+06                 8.108270
5           Bronx    6      5.689661e+06                 3.796290
6           Bronx    7      5.256189e+06                 4.028097
7           Bronx    8      4.755351e+06                 3.889064
8           Bronx    9      4.606810e+06                 6.287741
9           Bronx   10      4.781924e+06                 5.053522
10          Bronx   11      4.391277e+06                 3.523584
11          Bronx   12      4.699306e+06                 3.442847
12       Brooklyn    1      3.930719e+07                12.595086
13       Brooklyn    2      3.972768e+07                12.595075
14       B

In [16]:
print("Pregunta de Negocio 2: Ingresos y propinas: ¿cómo varían los ingresos totales y el tip % por borough y mes? (Acorde a la zona de Dropoff)")

# query para ingresos y propinas por borough y mes en dropoff
query = """
with viajes_con_borough as (
    select 
        t.fare_amount,
        t.tip_percentage,
        dd.month as mes,
        dz.dropoff_borough
    from ny_taxi.taxis_gold.fct_trips as t
    join ny_taxi.taxis_gold.dim_zone_dropoff as dz using (dropoff_zone_sk)
    join ny_taxi.taxis_gold.dim_date_dropoff as dd using (dropoff_date_sk)
    where t.fare_amount >= 0 and dz.dropoff_borough is not null
)

select
    dropoff_borough,
    mes,
    sum(fare_amount) as ingresos_totales,
    avg(tip_percentage) as tip_porcentaje_promedio
from viajes_con_borough
group by dropoff_borough, mes
order by dropoff_borough, mes;
"""

# Ejecutar query y devolver DataFrame
df = pd.read_sql(query, conn)

# Ver resultados
print(df)

Pregunta de Negocio 2: Ingresos y propinas: ¿cómo varían los ingresos totales y el tip % por borough y mes? (Acorde a la zona de Dropoff)


C:\Users\john8\AppData\Local\Temp\ipykernel_25788\4168734427.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   DROPOFF_BOROUGH  MES  INGRESOS_TOTALES  TIP_PORCENTAJE_PROMEDIO
0            Bronx    1      1.732992e+07                 5.190860
1            Bronx    2      1.751470e+07                 9.880922
2            Bronx    3      2.007580e+07                 4.895399
3            Bronx    4      1.850746e+07                 5.245061
4            Bronx    5      1.916864e+07                 5.353877
5            Bronx    6      1.811336e+07                 5.504375
6            Bronx    7      1.760628e+07                 5.397887
7            Bronx    8      1.679811e+07                 5.739658
8            Bronx    9      1.554678e+07                 5.485766
9            Bronx   10      1.558322e+07                 5.720034
10           Bronx   11      1.460026e+07                 6.174936
11           Bronx   12      1.560815e+07                 5.327784
12        Brooklyn    1      9.215182e+07                17.481327
13        Brooklyn    2      9.210284e+07                13.79

In [ ]:
print("Pregunta 3: Velocidad y congestión: promedio de mph por franja horaria y borough (viajes diurnos vs. nocturnos). (Acorde a la zona de Pickup)")

# query para velocidad y congestión por franja horaria y borough en pickup
query = """
with viajes_clasificados as (
    select
        dz.pickup_borough,
        dt.hour,
        t.avg_speed_mph,
        case 
            when dt.hour between 6 and 18 then 'diurno'
            else 'nocturno'
        end as franja
    from ny_taxi.taxis_gold.fct_trips as t
    join ny_taxi.taxis_gold.dim_zone_pickup as dz using (pickup_zone_sk)
    join ny_taxi.taxis_gold.dim_time_pickup as dt using (pickup_time_sk)
    where t.avg_speed_mph > 0 and dz.pickup_borough is not null
)

select
    pickup_borough,
    franja,
    avg(avg_speed_mph) as avg_speed_mph_franja
from viajes_clasificados
group by pickup_borough, franja
order by pickup_borough, franja;
"""

# Ejecutar query y devolver DataFrame
df = pd.read_sql(query, conn)

# Ver resultados
print(df)

Pregunta 3: Velocidad y congestión: promedio de mph por franja horaria y borough (viajes diurnos vs. nocturnos). (Acorde a la zona de Pickup)


C:\Users\john8\AppData\Local\Temp\ipykernel_25788\3729858942.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   PICKUP_BOROUGH    FRANJA  AVG_SPEED_MPH_FRANJA
0           Bronx    diurno             12.659810
1           Bronx  nocturno             15.819488
2        Brooklyn    diurno             11.409301
3        Brooklyn  nocturno             13.721568
4             EWR    diurno             23.587869
5             EWR  nocturno             24.861647
6       Manhattan    diurno              9.848338
7       Manhattan  nocturno             12.413541
8          Queens    diurno             17.486334
9          Queens  nocturno             22.493090
10  Staten Island    diurno             20.700636
11  Staten Island  nocturno             25.836062
12        Unknown    diurno             10.722284
13        Unknown  nocturno             13.671384


In [18]:
print("Pregunta 3 (Dropoff): Velocidad y congestión: promedio de mph por franja horaria y borough (viajes diurnos vs. nocturnos). (Acorde a la zona de Dropoff)")

# query para velocidad y congestión por franja horaria y borough en dropoff
query = """
with viajes_clasificados as (
    select
        dz.dropoff_borough,
        dt.hour,
        t.avg_speed_mph,
        case 
            when dt.hour between 6 and 18 then 'diurno'
            else 'nocturno'
        end as franja
    from ny_taxi.taxis_gold.fct_trips as t
    join ny_taxi.taxis_gold.dim_zone_dropoff as dz using (dropoff_zone_sk)
    join ny_taxi.taxis_gold.dim_time_dropoff as dt using (dropoff_time_sk)
    where t.avg_speed_mph > 0 and dz.dropoff_borough is not null
)

select
    dropoff_borough,
    franja,
    avg(avg_speed_mph) as avg_speed_mph_franja
from viajes_clasificados
group by dropoff_borough, franja
order by dropoff_borough, franja;
"""

# Ejecutar query y devolver DataFrame
df = pd.read_sql(query, conn)

# Ver resultados
print(df)


Pregunta 3 (Dropoff): Velocidad y congestión: promedio de mph por franja horaria y borough (viajes diurnos vs. nocturnos). (Acorde a la zona de Dropoff)


C:\Users\john8\AppData\Local\Temp\ipykernel_25788\175565364.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   DROPOFF_BOROUGH    FRANJA  AVG_SPEED_MPH_FRANJA
0            Bronx    diurno             15.290128
1            Bronx  nocturno             20.865660
2         Brooklyn    diurno             12.995067
3         Brooklyn  nocturno             15.677464
4              EWR    diurno             27.783343
5              EWR  nocturno             31.757169
6        Manhattan    diurno              9.781775
7        Manhattan  nocturno             12.311956
8           Queens    diurno             17.947704
9           Queens  nocturno             17.951090
10   Staten Island    diurno             25.612771
11   Staten Island  nocturno             29.802124
12         Unknown    diurno             10.477197
13         Unknown  nocturno             13.135498


In [19]:
print("Pregunta 4: Duración del viaje: percentiles (p50/p90) de duración por PULocationID (pickup)")

# query para percentiles de tiempo de viaje por pickup location id
query = """
select
    dzp.pickup_location_id,
    dzp.pickup_borough,
    dzp.pickup_zone,
    percentile_cont(0.5) within group (order by t.trip_duration_seconds) as p50_duration_sec,
    percentile_cont(0.9) within group (order by t.trip_duration_seconds) as p90_duration_sec
from ny_taxi.taxis_gold.fct_trips as t
join ny_taxi.taxis_gold.dim_zone_pickup as dzp
    using (pickup_zone_sk)
group by dzp.pickup_location_id, dzp.pickup_borough, dzp.pickup_zone
order by dzp.pickup_location_id;
"""

# Ejecutar query y devolver DataFrame
df = pd.read_sql(query, conn)

# Ver resultados
print(df)

Pregunta 4: Duración del viaje: percentiles (p50/p90) de duración por PULocationID (pickup)


C:\Users\john8\AppData\Local\Temp\ipykernel_25788\563305843.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


     PICKUP_LOCATION_ID PICKUP_BOROUGH  \
0                     1            EWR   
1                     2         Queens   
2                     3          Bronx   
3                     4      Manhattan   
4                     5  Staten Island   
5                     6  Staten Island   
6                     7         Queens   
7                     8         Queens   
8                     9         Queens   
9                    10         Queens   
10                   11       Brooklyn   
11                   12      Manhattan   
12                   13      Manhattan   
13                   14       Brooklyn   
14                   15         Queens   
15                   16         Queens   
16                   17       Brooklyn   
17                   18          Bronx   
18                   19         Queens   
19                   20          Bronx   
20                   21       Brooklyn   
21                   22       Brooklyn   
22                   23  Staten Is

In [26]:
print("Pregunta 5: Elasticidad temporal: distribución de viajes por día de semana y hora; ¿cuáles son las horas pico?")

# query para distribución de viajes por día de semana y hora
query = """
select
    dd.day_of_week,
    dt.hour as pickup_hour,
    count(*) as total_viajes
from ny_taxi.taxis_gold.fct_trips as t
join ny_taxi.taxis_gold.dim_date_pickup as dd using (pickup_date_sk)
join ny_taxi.taxis_gold.dim_time_pickup as dt using (pickup_time_sk)
group by dd.day_of_week, dt.hour
order by dd.day_of_week, total_viajes desc;
"""

# Ejecutar query y devolver DataFrame
df = pd.read_sql(query, conn)

# Ver resultados
print(df)

print("Acorde a los resultados obtenidos, se pueden identificar las horas pico son las siguientes: \nLunes: 18, 19\nMartes: 18, 19\nMiércoles: 18, 19\nJueves: 18, 19\nViernes: 18, 19\nSábado: 19, 18\nDomingo: 0, 18")

Pregunta 5: Elasticidad temporal: distribución de viajes por día de semana y hora; ¿cuáles son las horas pico?


C:\Users\john8\AppData\Local\Temp\ipykernel_25788\3992710429.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


    DAY_OF_WEEK  PICKUP_HOUR  TOTAL_VIAJES
0       domingo            0       6189671
1       domingo           18       6098580
2       domingo           14       6079929
3       domingo           17       6015603
4       domingo           13       5984074
5       domingo           15       5948319
6       domingo           12       5896297
7       domingo           16       5847193
8       domingo           19       5500982
9       domingo            1       5462450
10      domingo           11       5389685
11      domingo           20       4979955
12      domingo           21       4708392
13      domingo           10       4669596
14      domingo            2       4273585
15      domingo           22       4159366
16      domingo            9       3450803
17      domingo            3       3292095
18      domingo           23       3253452
19      domingo            8       2314584
20      domingo            4       2110253
21      domingo            7       1547340
22      dom